In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

In [2]:
import numpy as np 
from tqdm import tqdm
import cv2
import os
import shutil
import itertools
import imutils
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools

from keras import layers, models
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.layers.core import Activation, Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import backend as K

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications import ResNet101
from keras.applications import DenseNet201, DenseNet169, DenseNet121
from keras.applications import InceptionV3
from keras.applications import InceptionResNetV2
from keras.applications import Xception
from keras.applications import MobileNetV2
from keras.applications import MobileNet
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping
import pandas as pd

from sklearn.svm import SVC

init_notebook_mode(connected=True)
RANDOM_SEED = 123

Using TensorFlow backend.


In [3]:
def load_data(dir_path, img_size=(100, 100)):
    X = []
    y = []
    i = 0
    labels = dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith('.'):
                    img = cv2.imread(dir_path + path + '/' + file)
                    X.append(img)
                    y.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print('{} images loaded from {} directory.'.format(len(X), dir_path))
    return X, y, labels
    

In [4]:
TRAIN_DIR = 'TRAIN/'
TEST_DIR = 'TEST/'
VAL_DIR = 'VAL/'
IMG_SIZE = (224, 224)

X_train, y_train, labels = load_data(TRAIN_DIR, IMG_SIZE)
X_test, y_test, _ = load_data(TEST_DIR, IMG_SIZE)
X_val, y_val, _ = load_data(VAL_DIR, IMG_SIZE)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.27it/s]
C:\Users\zeyne\miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:15: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray

 50%|██████████████████████████████████████████                                          | 1/2 [00:00<00:00,  7.56it/s]

242 images loaded from TRAIN/ directory.
19 images loaded from TEST/ directory.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.86it/s]

92 images loaded from VAL/ directory.


In [5]:
def preprocess_imgs(set_name, img_size):
    
    # Resize and apply VGG-15 preprocessing
    set_new = []
    for img in set_name:
        img = cv2.resize(
            img,
            dsize=img_size,
            interpolation=cv2.INTER_CUBIC
        )
        set_new.append(preprocess_input(img))
    return np.array(set_new)

In [6]:
X_train_prep = preprocess_imgs(set_name=X_train, img_size=IMG_SIZE)
X_test_prep = preprocess_imgs(set_name=X_test, img_size=IMG_SIZE)
X_val_prep = preprocess_imgs(set_name=X_val, img_size=IMG_SIZE)

# CNN Pre-trained Models

## Data Augmentation

In [7]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    color_mode='rgb',
    target_size=IMG_SIZE,
    batch_size=8,
    class_mode='binary',
    seed=RANDOM_SEED
)

validation_generator = test_datagen.flow_from_directory(
    VAL_DIR,
    color_mode='rgb',
    target_size=IMG_SIZE,
    batch_size=8,
    class_mode='binary',
    seed=RANDOM_SEED
)

Found 242 images belonging to 2 classes.
Found 92 images belonging to 2 classes.


In [8]:
# loading base model
def load_feature_extractor():
    base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=IMG_SIZE + (3,))
    return base_model

In [9]:
model = load_feature_extractor()
#model.summary()

In [10]:
def extract_features(img_data):
    
    features = []
    for img in img_data:
        img = np.expand_dims(img, axis=0)
        feature = model.predict(img)
        feature_np = np.array(feature)
        features.append(feature_np.flatten())
    
    feature_list_np = np.array(features)
    
    return feature_list_np
        

In [11]:
X_train_ftr = extract_features(X_train_prep)
y_train_ftr = y_train
X_test_ftr = extract_features(X_test_prep)
y_test_ftr = y_test
X_val_ftr = extract_features(X_val_prep)
y_val_ftr = y_val

In [12]:
X_test_ftr.shape

(19, 62720)

### SVM

In [13]:
from sklearn.svm import SVC
classifier_SVM = SVC(kernel = 'rbf', random_state = 0)
classifier_SVM.fit(X_train_ftr, y_train_ftr)

SVC(random_state=0)

In [14]:
train_acc_SVM = classifier_SVM.score(X_train_ftr, y_train_ftr)
val_acc_SVM = classifier_SVM.score(X_val_ftr, y_val_ftr)
test_acc_SVM = classifier_SVM.score(X_test_ftr, y_test_ftr)

In [15]:
print(np.round(train_acc_SVM, 4))
print(np.round(val_acc_SVM, 4))
print(np.round(test_acc_SVM, 4))

0.9752
0.9674
0.9474


In [16]:
y_pred_SVM = classifier_SVM.predict(X_test_ftr)

In [17]:
from sklearn import metrics
def print_performance_metrics(test_labels,predict):
    print('F1 Score:', np.round(metrics.f1_score(test_labels, predict,
                                               average='weighted'),4))
    print('Cohen Kappa Score:', np.round(metrics.cohen_kappa_score(test_labels, predict),4))
    print('Recall:', np.round(metrics.recall_score(test_labels, predict,
                                               average='weighted'),4))
    print('Precision:', np.round(metrics.precision_score(test_labels, predict,average='weighted'),4))
    print('Accuracy:', np.round(metrics.accuracy_score(test_labels, predict),4))
    print('ROC Area:', np.round(metrics.roc_auc_score(test_labels, predict),4))
    print('Recall:', np.round(metrics.recall_score(test_labels, predict,
                                               average='weighted'),4))
    print('Matthews Corrcoef:', np.round(metrics.matthews_corrcoef(test_labels, predict),4)) 
    print('\t\tClassification Report:\n', metrics.classification_report(test_labels, predict))

print_performance_metrics(y_test,y_pred_SVM)

F1 Score: 0.9474
Cohen Kappa Score: 0.895
Recall: 0.9474
Precision: 0.9526
Accuracy: 0.9474
ROC Area: 0.95
Recall: 0.9474
Matthews Corrcoef: 0.9
		Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.90      0.95        10
           1       0.90      1.00      0.95         9

    accuracy                           0.95        19
   macro avg       0.95      0.95      0.95        19
weighted avg       0.95      0.95      0.95        19



### RANDOM FOREST

In [18]:
from sklearn.ensemble import RandomForestClassifier
classifier_RF = RandomForestClassifier(n_estimators = 800, criterion = 'entropy',  random_state = 0)
classifier_RF.fit(X_train_ftr, y_train_ftr)

RandomForestClassifier(criterion='entropy', n_estimators=800, random_state=0)

In [19]:
train_acc_RF = classifier_RF.score(X_train_ftr, y_train_ftr)
print(np.round(train_acc_RF, 4))
val_acc_RF = classifier_RF.score(X_val_ftr, y_val_ftr)
print(np.round(val_acc_RF, 4))
test_acc_RF = classifier_RF.score(X_test_ftr, y_test_ftr)
print(np.round(test_acc_RF, 4))

1.0
0.9783
0.9474


In [20]:
y_pred_RF = classifier_RF.predict(X_test_ftr)
print_performance_metrics(y_test,y_pred_RF)

F1 Score: 0.9474
Cohen Kappa Score: 0.895
Recall: 0.9474
Precision: 0.9526
Accuracy: 0.9474
ROC Area: 0.95
Recall: 0.9474
Matthews Corrcoef: 0.9
		Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.90      0.95        10
           1       0.90      1.00      0.95         9

    accuracy                           0.95        19
   macro avg       0.95      0.95      0.95        19
weighted avg       0.95      0.95      0.95        19



### ADABOOST

In [21]:
from sklearn.ensemble import AdaBoostClassifier
classifier_AB = AdaBoostClassifier(n_estimators = 100)
classifier_AB.fit(X_train_ftr, y_train_ftr)

AdaBoostClassifier(n_estimators=100)

In [22]:
train_acc_AB = classifier_AB.score(X_train_ftr, y_train_ftr)
print(train_acc_AB)
val_acc_AB = classifier_AB.score(X_val_ftr, y_val_ftr)
print(val_acc_AB)
test_acc_AB = classifier_AB.score(X_test_ftr, y_test_ftr)
print(test_acc_AB)

1.0
1.0
1.0


In [23]:
y_pred_AB = classifier_AB.predict(X_test_ftr)
print_performance_metrics(y_test,y_pred_AB)

F1 Score: 1.0
Cohen Kappa Score: 1.0
Recall: 1.0
Precision: 1.0
Accuracy: 1.0
ROC Area: 1.0
Recall: 1.0
Matthews Corrcoef: 1.0
		Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9

    accuracy                           1.00        19
   macro avg       1.00      1.00      1.00        19
weighted avg       1.00      1.00      1.00        19



### KNN

In [24]:
from sklearn.neighbors import KNeighborsClassifier
classifier_kNN = KNeighborsClassifier(n_neighbors = 5, algorithm='ball_tree', leaf_size=30)
classifier_kNN.fit(X_train_ftr, y_train_ftr)

KNeighborsClassifier(algorithm='ball_tree')

In [25]:
train_acc_kNN = classifier_kNN.score(X_train_ftr, y_train_ftr)
print(train_acc_kNN)
val_acc_kNN = classifier_kNN.score(X_val_ftr, y_val_ftr)
print(val_acc_kNN)
test_acc_kNN = classifier_kNN.score(X_test_ftr, y_test_ftr)
print(test_acc_kNN)

0.8471074380165289
0.8478260869565217
0.8421052631578947


In [26]:
y_pred_kNN = classifier_kNN.predict(X_test_ftr)
print_performance_metrics(y_test,y_pred_kNN)

F1 Score: 0.8412
Cohen Kappa Score: 0.6816
Recall: 0.8421
Precision: 0.8451
Accuracy: 0.8421
ROC Area: 0.8389
Recall: 0.8421
Matthews Corrcoef: 0.6854
		Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.90      0.86        10
           1       0.88      0.78      0.82         9

    accuracy                           0.84        19
   macro avg       0.85      0.84      0.84        19
weighted avg       0.85      0.84      0.84        19



### XGBOOST

In [27]:
import xgboost as xgb
classifier_xgb = xgb.XGBClassifier(n_estimators = 300)
classifier_xgb.fit(X_train_ftr, y_train_ftr)

C:\Users\zeyne\miniconda3\envs\tensorflow\lib\site-packages\xgboost\sklearn.py:888: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].



[00:05:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=300, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [28]:
train_acc_xgb = classifier_xgb.score(X_train_ftr, y_train_ftr)
print(train_acc_xgb)
val_acc_xgb = classifier_xgb.score(X_val_ftr, y_val_ftr)
print(val_acc_xgb)
test_acc_xgb = classifier_xgb.score(X_test_ftr, y_test_ftr)
print(test_acc_xgb)

1.0
1.0
1.0


In [29]:
y_pred_xgb = classifier_xgb.predict(X_test_ftr)
print_performance_metrics(y_test,y_pred_xgb)

F1 Score: 1.0
Cohen Kappa Score: 1.0
Recall: 1.0
Precision: 1.0
Accuracy: 1.0
ROC Area: 1.0
Recall: 1.0
Matthews Corrcoef: 1.0
		Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9

    accuracy                           1.00        19
   macro avg       1.00      1.00      1.00        19
weighted avg       1.00      1.00      1.00        19



### ANN

In [30]:
def model_ANN():
    model = Sequential()
    model.add(Input(shape = (last_layer_shape[1], last_layer_shape[2], last_layer_shape[3])))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(Dropout(0.25))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model


In [31]:
model_ANN = model_ANN()


model_ANN.summary()
model_ANN.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(lr=1e-4),
    metrics=['accuracy'])

model_ANN.summary()


NameError: name 'Input' is not defined

In [ ]:
history_ANN = model_ANN.fit(
    X_train_ftr, y_train_ftr, 
    epochs=40, 
    verbose=1, 
    validation_data=(X_val_ftr,y_val_ftr))